In [ ]:
import pandas as pd
import numpy as np
import torch
from torch import nn
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split
import os
import glob
import shutil


from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
)

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("DEVICE:", DEVICE)

# ===============================
# 0. 긴 텍스트 chunk 함수 (문자 기준)
# ===============================

def split_into_chunks(text, chunk_chars=4000):
    """
    긴 문자열을 chunk_chars 글자 단위로 잘라 리스트로 반환.
    ex) chunk_chars=4000이면 0~3999, 4000~7999, ... 이런 식으로 나뉨.
    """
    text = str(text)
    return [text[i:i + chunk_chars] for i in range(0, len(text), chunk_chars)]

# ===============================
# 1. Dataset 정의
# ===============================

class NewsLongformerDataset(Dataset):
    def __init__(self, df, referrers, tokenizer, max_length=4096):
        """
        df: index = article_id, columns = ['content', <referrer1> ...]
        referrers: 라벨로 쓸 referrer 이름 리스트
        """
        self.df = df.reset_index()
        self.referrers = referrers
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        text = str(row["content"])

        enc = self.tokenizer(
            text,
            truncation=True,
            padding="max_length",
            max_length=self.max_length,
            return_tensors="pt",
        )

        input_ids = enc["input_ids"].squeeze(0)
        attention_mask = enc["attention_mask"].squeeze(0)

        # Longformer: 첫 토큰 global attention
        global_attention_mask = torch.zeros_like(attention_mask)
        global_attention_mask[0] = 1

        labels = torch.tensor(
            [row[r] for r in self.referrers],
            dtype=torch.float32
        )

        return {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "global_attention_mask": global_attention_mask,
            "labels": labels,
        }


# ===============================
# 2. Soft Label용 Trainer
# ===============================

class SoftLabelTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        # labels: soft label (확률 분포)
        labels = inputs.pop("labels")  # (B, num_labels)
        outputs = model(**inputs)
        logits = outputs.logits                    # (B, num_labels)
        log_probs = torch.nn.functional.log_softmax(logits, dim=-1)

        # soft label cross-entropy: - sum_k y_k log p_k
        loss = -(labels * log_probs).sum(dim=-1).mean()

        if return_outputs:
            return loss, outputs
        return loss


# ===============================
# 3. metrics 계산 함수
# ===============================

def compute_metrics(eval_pred):
    """
    Trainer가 eval 시 호출.
    predictions: logits (numpy)
    label_ids: soft label 분포 (numpy)
    """
    logits, labels = eval_pred
    
    probs = torch.softmax(torch.tensor(logits), dim=-1).numpy()

    # soft label에서 argmax로 "정답 referrer" 인덱스
    # argmax 반환 값은 class 번호만 나옴
    true_idx = labels.argmax(axis=-1)
    pred_idx = probs.argmax(axis=-1)
    accuracy = (true_idx == pred_idx).mean()

    # 1) Cross-Entropy (soft label 기준)
    eps = 1e-12
    ce = -(labels * np.log(probs + eps)).sum(axis=-1).mean()

    # 2) MSE / Brier score
    mse = ((probs - labels) ** 2).mean()

    return {
        "accuracy": float(accuracy),
        "cross_entropy": float(ce),
        "mse": float(mse),
    }



# ===============================
# 4. 데이터 준비 + 학습 함수
# ===============================

def train_model(
    df_path: str,
    batch_size: int = 1,
    lr: float = 2e-5,
    num_epochs: int = 3,
    max_length: int = 4096,
    min_ref_count: int = 100,
    output_dir: str = "./longformer_platform_trainer",
    chunk_chars: int = 4000, 
):
    """
    df_path: news_merged.xlsx 같은 파일 경로
    return: (model, tokenizer, referrers, trainer)
    """

    # ---------- 4-1. 데이터 로드 ----------
    # ---------- 4-1. 데이터 로드 ----------
    df = pd.read_excel(df_path)

    needed_cols = ["article_id", "content", "views_total", "referrer", "share"]
    for c in needed_cols:
        if c not in df.columns:
            raise ValueError(f"필수 컬럼 {c} 이(가) 데이터에 없습니다.")

    # ---------- 4-1-1. referrer 가공 (복사해서 사용) ----------
    # 원본 보호용 (엑셀은 건드리지 않고, 메모리 상 df만 변경)
    df = df.copy()

    # 1) Facebook, namu.wiki, 야후 → '기타'로 그룹화한 새 컬럼 만들기
    df["referrer_grouped"] = df["referrer"].replace({
        "Facebook": "기타",
        "namu.wiki": "기타",
        "야후": "기타",
    })

    # 2) referrer_grouped 기준으로 개수 줄이기
    #    네이버: 10000, 네이버 블로그: 7000, Google: 5000
    max_per_class = {
        "네이버": 10000,
        "네이버 블로그": 7000,
        "Google": 5000,
        # 기타, Daum 등은 제한 없음
    }

    balanced_parts = []
    for label, group in df.groupby("referrer_grouped"):
        if label in max_per_class and len(group) > max_per_class[label]:
            sampled = group.sample(max_per_class[label], random_state=42)
            balanced_parts.append(sampled)
        else:
            balanced_parts.append(group)

    df = pd.concat(balanced_parts).sample(frac=1, random_state=42).reset_index(drop=True)

    # 이후 코드에서는 referrer_grouped를 referrer로 사용하도록 교체
    df["referrer"] = df["referrer_grouped"]
    df = df.drop(columns=["referrer_grouped"])

    # 가공 후 분포 확인해보고 싶으면 이거 잠깐 찍어봐도 됨
    print(df["referrer"].value_counts())


    # ---------- 4-2. referrer 등장 횟수 필터 ----------
    ref_counts = df["referrer"].value_counts()
    valid_referrers = ref_counts[ref_counts >= min_ref_count].index.tolist()
    print("사용할 referrer 수:", len(valid_referrers))
    print("예시 referrer:", valid_referrers[:10])

    df = df[df["referrer"].isin(valid_referrers)].copy()

    # ---------- 4-3. share -> 0~1 ----------
    df["share_frac"] = df["share"] / 100.0

    # ---------- 4-4. 조회수 가중 soft label ----------
    # y(article, referrer) = sum(views * share_frac) / sum(views)
    grouped = (
        df
        .groupby(["article_id", "referrer"])
        .apply(lambda g: (g["views_total"] * g["share_frac"]).sum() / g["views_total"].sum())
        .reset_index(name="soft_label")
    )

    # ---------- 4-5. article_id 기준 referrer 분포 pivot ----------
    label_mat = (
        grouped
        .pivot(index="article_id", columns="referrer", values="soft_label")
        .fillna(0.0)
    )

    # 정규화 (합 = 1)
    row_sum = label_mat.sum(axis=1)
    row_sum[row_sum == 0.0] = 1.0
    label_mat = label_mat.div(row_sum, axis=0)

    # ---------- 4-6. 기사 본문 붙이기 ----------
    article_text = (
        df
        .drop_duplicates("article_id")
        .set_index("article_id")[["content"]]
    )

    dataset_df = article_text.join(label_mat, how="inner").dropna()

    referrer_labels = list(label_mat.columns)
    num_labels = len(referrer_labels)

    print("최종 데이터 크기:", dataset_df.shape)
    print("라벨(referrer) 개수:", num_labels)

    label_array = dataset_df[referrer_labels].values
    eps = 1e-12
    entropy = -(label_array * np.log(label_array + eps)).sum(axis=1).mean()
    print("평균 라벨 엔트로피 H(y):", entropy)


    chunk_chars = 4000
    
    expanded_rows = []
    for article_id, row in dataset_df.iterrows():
        chunks = split_into_chunks(row["content"], chunk_chars=chunk_chars)
        for ci, chunk in enumerate(chunks):
            new_row = row.copy()
            new_row["content"] = chunk
            new_row["chunk_id"] = ci
            new_row["article_id"] = article_id
            expanded_rows.append(new_row)

    tokenizer = AutoTokenizer.from_pretrained("monologg/kobigbird-bert-base")

    # chunk 1000 토큰 이하 없애기
    filtered_rows = []
    for row in expanded_rows:
        tokens = tokenizer.encode(row["content"], add_special_tokens=True)
        if len(tokens) >= 1000:
            filtered_rows.append(row)


    dataset_df = pd.DataFrame(filtered_rows).set_index("article_id")
    print("chunk 확장 후 데이터 크기:", dataset_df.shape)




    # ---------- 4-7. train / val split 비율 동일하게 나눔----------
    dataset_df["major_referrer"] = dataset_df[referrer_labels].idxmax(axis=1)

    # 2) Stratified split
    train_df, val_df = train_test_split(
        dataset_df,
        test_size=0.2,
        random_state=42,
        stratify=dataset_df["major_referrer"],   # ← 핵심!
    )

    # ---------- 4-8. 토크나이저 & Dataset ----------

    train_dataset = NewsLongformerDataset(train_df, referrer_labels, tokenizer, max_length=max_length)
    val_dataset   = NewsLongformerDataset(val_df,   referrer_labels, tokenizer, max_length=max_length)



    # ---------- 4-9. 모델 ----------
    model = AutoModelForSequenceClassification.from_pretrained(
        "monologg/kobigbird-bert-base",
        num_labels=num_labels,
    )
    model.to(DEVICE)

    # ---------- 4-10. TrainingArguments ----------
    training_args = TrainingArguments(
        output_dir=output_dir,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        learning_rate=lr,
        num_train_epochs=num_epochs,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        logging_steps=50,
        load_best_model_at_end=True,
        metric_for_best_model="eval_loss",  # eval_loss 기준으로 best model
        report_to="none",
#         gradient_accumulation_steps=4,
        save_safetensors=True,
    )

    # ---------- 4-11. Trainer ----------
    trainer = SoftLabelTrainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics,
    )

    # ---------- 4-12. 학습 ----------
    trainer.train()

    # ---------- 4-13. 최종 평가 ----------
    eval_result = trainer.evaluate()
    print("\n최종 평가 결과:")
    for k, v in eval_result.items():
        print(f"{k}: {v}")
    

        # ---------- 4-14. best / last checkpoint 정리 ----------

    # 1) best checkpoint 경로 (eval_loss 기준)
    best_ckpt = trainer.state.best_model_checkpoint
    print("Best checkpoint:", best_ckpt)

    # 2) output_dir 아래의 모든 checkpoint-* 디렉토리 목록
    ckpt_dirs = sorted(
        [
            p
            for p in glob.glob(os.path.join(output_dir, "checkpoint-*"))
            if os.path.isdir(p)
        ],
        key=lambda p: int(p.split("-")[-1])  # "checkpoint-1234"의 숫자 부분 기준 정렬
    )

    last_ckpt = ckpt_dirs[-1] if ckpt_dirs else None
    print("Last checkpoint:", last_ckpt)

    # 3) best 모델을 따로 저장 (best_model 디렉토리)
    if best_ckpt is not None:
        best_model_dir = os.path.join(output_dir, "best_model")
        print("Saving best model to:", best_model_dir)
        # best checkpoint에서 다시 로드해서 저장
        best_model = AutoModelForSequenceClassification.from_pretrained(
            best_ckpt,
            num_labels=num_labels,
        )
        best_model.save_pretrained(best_model_dir)
        tokenizer.save_pretrained(best_model_dir)

    # 4) 마지막 checkpoint 기준 모델도 따로 저장하고 싶다면 (선택)
    if last_ckpt is not None:
        last_model_dir = os.path.join(output_dir, "last_model")
        print("Saving last model to:", last_model_dir)
        last_model = AutoModelForSequenceClassification.from_pretrained(
            last_ckpt,
            num_labels=num_labels,
        )
        last_model.save_pretrained(last_model_dir)
        tokenizer.save_pretrained(last_model_dir)

    # 5) checkpoint 디렉토리는 "best + last"만 남기고 나머지 삭제
    for ckpt in ckpt_dirs:
        if ckpt != best_ckpt and ckpt != last_ckpt:
            print("Removing old checkpoint:", ckpt)
            shutil.rmtree(ckpt)

    return trainer.model, tokenizer, referrer_labels, trainer


/home/u4005/.venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-12-05 15:46:52.215712: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-12-05 15:46:52.714604: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


DEVICE: cuda


In [2]:
# ===============================
# 5. 예측 함수 (확률 10% 미만 출력 X)
# ===============================

def predict_platform_distribution(
    text: str,
    model,
    tokenizer,
    referrers,
    max_length: int = 4096,
    threshold: float = 0.1,
):
    model.eval()
    with torch.no_grad():
        enc = tokenizer(
            text,
            truncation=True,
            padding="max_length",
            max_length=max_length,
            return_tensors="pt",
        )
        input_ids = enc["input_ids"].to(DEVICE)
        attention_mask = enc["attention_mask"].to(DEVICE)

        global_attention_mask = torch.zeros_like(attention_mask).to(DEVICE)
        global_attention_mask[:, 0] = 1

        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            global_attention_mask=global_attention_mask,
        )
        probs = torch.softmax(outputs.logits, dim=-1)[0].cpu().numpy()
        
    result = {
        ref: float(p)
        for ref, p in sorted(
            zip(referrers, probs),
            key=lambda x: x[1],
            reverse=True
        )
        if p >= threshold
    }
    return result


# ===============================
# 6. 사용 예시
# ===============================

if __name__ == "__main__":
    excel_path = "../AI_model/신문 데이터/news_merged_grouped_balanced.xlsx"

    model, tokenizer, referrers, trainer = train_model(
        df_path=excel_path,
        batch_size=1,
        lr=1e-5,
        num_epochs=3,
        max_length=4096,
        min_ref_count=100,
        output_dir="./longformer_3epoch_new",
    )


referrer
네이버        10000
네이버 블로그     7000
Google      5000
기타          4244
Daum        2278
AI 검색엔진     1244
Bing        1189
Name: count, dtype: int64
사용할 referrer 수: 7
예시 referrer: ['네이버', '네이버 블로그', 'Google', '기타', 'Daum', 'AI 검색엔진', 'Bing']
최종 데이터 크기: (1618, 8)
라벨(referrer) 개수: 7
평균 라벨 엔트로피 H(y): 0.9347150455321372
chunk 확장 후 데이터 크기: (2220, 9)


Some weights of BigBirdForSequenceClassification were not initialized from the model checkpoint at monologg/kobigbird-bert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/u4005/.venv/lib/python3.8/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/tmp/ipykernel_3219298/1456304933.py:328: FutureWarning: `tokenize

Epoch,Training Loss,Validation Loss,Accuracy,Cross Entropy,Mse
1,1.788100,1.758769,0.488739,1.758769,0.037456
2,1.816900,1.732279,0.479730,1.732279,0.036327
3,1.627500,1.749698,0.475225,1.749698,0.037433



최종 평가 결과:
eval_loss: 1.732278823852539
eval_accuracy: 0.4797297297297297
eval_cross_entropy: 1.7322789430618286
eval_mse: 0.036326758563518524
eval_runtime: 53.4297
eval_samples_per_second: 8.31
eval_steps_per_second: 8.31
epoch: 3.0
Best checkpoint: ./longformer_3epoch_new/checkpoint-3552
Last checkpoint: ./longformer_3epoch_new/checkpoint-5328
Saving best model to: ./longformer_3epoch_new/best_model
Saving last model to: ./longformer_3epoch_new/last_model


NameError: name 'last_model' is not defined

In [ ]:
!pip install deepspeed


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 31.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for deepspeed: filename=deepspeed-0.18.2-py3-none-any.whl size=1763296 sha256=90738e4def1dccf7b0c9c5cc1673bd517bdd7f6333412f8f936e6f7ac065268c
  Stored in directory: /home/u4005/.cache/pip/wheels/30/1d/94/a05d95c116aa0a299a3494cd8a2bae3d3d5f2d5ba7ea917ea5
Successfully built deepspeed
